# Categorizing zoo animal species by microbiome

## 1. Setup
### 1.1 Libraries

In [1]:
import pandas as pd
import numpy as np
import altair as alt

# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Turn only off for presentation purposes
import warnings
warnings.filterwarnings("ignore")

### 1.2 Data import
Data has been preprocessed in R. We removed all animal species with less than 20 probes.

In [2]:
# Read data 
df = pd.read_csv('data/data_clean.csv')
metadata = df.iloc[:,:9].drop_duplicates().sort_values(['Familie','Gattung','Art']).reset_index(drop=True)
metadata_familie = metadata[['Familie','Diet','digestion']].drop_duplicates().reset_index(drop=True)
metadata_gattung = metadata[['Gattung','Diet','digestion']].drop_duplicates().reset_index(drop=True)
# Identifying zoo and individuals from index name
df.insert(0, 'Zoo', df['index'].str[:3])
df.insert(1, 'AnimalID', df['index'].str[7:])

### 1.3 Helper functions

In [3]:
def train_dev_test_split(df, y_name="", test_size=0.2, random_state=42):
    if y_name != "":
        # Check if stratification is possible
        can_stratify = df[y_name].value_counts().min() > 1
        
        # If stratification is possible, use it
        if can_stratify:
            train, test = train_test_split(
                df, test_size=test_size/(1-test_size), 
                random_state=random_state, stratify=df[y_name]
            )
        # If not, do a simple split without stratification
        else:
            train, test = train_test_split(
                df, test_size=test_size/(1-test_size), 
                random_state=random_state
            )
        
        # Define input and output variables
        X_train = train.iloc[:,12:]
        if y_name != 'Art':
            X_train = X_train.drop([y_name], axis=1)
        y_train = train[y_name]

        X_test = test.iloc[:,12:]
        if y_name != 'Art':
            X_test = X_test.drop([y_name], axis=1)
        y_test = test[y_name]
        
        return X_train, y_train, X_test, y_test
    
    else:
        # Check if stratification is possible
        can_stratify = df['Art'].value_counts().min() > 1
        
        # If stratification is possible, use it
        if can_stratify:
            train, test = train_test_split(
                df, test_size=test_size, 
                random_state=random_state, stratify=df['Art']
            )
        # If not, do a simple split without stratification
        else:
            train, test = train_test_split(
                df, test_size=test_size, 
                random_state=random_state
            )
        
        return train, test

# One-hot encoded data
def one_hot_encoding(df, Art):
    # One-hot encoding of column
    df_Art = pd.get_dummies(df.Art)
    # Join with dummy data
    df_tmp = df.iloc[:,:-1].join(df_Art[Art])
    # Split data
    X_train, y_train, X_dev, y_dev, X_test, y_test = train_dev_test_split(df_tmp, Art)
    return X_train, y_train, X_dev, y_dev, X_test, y_test

# Find best parameters using GridSearchCV for logistic regression
def lr_best_model(X_train, y_train):
    # Define multiple hyperparameter grids to search over
    param_grids = [
        {
            'penalty': ['l1', 'l2'],  # l1 and l2 penalties
            'C': [0.01, 0.1, 1.0, 10.0, 100.0],
            'solver': ['liblinear'],  # liblinear supports l1 and l2
            'max_iter': [100, 500, 1000],
        },
        {
            'penalty': ['l2', 'none'],  # l2 and none penalties
            'C': [0.01, 0.1, 1.0, 10.0, 100.0],
            'solver': ['newton-cg', 'lbfgs', 'sag'],  # these solvers support l2 and none
            'max_iter': [100, 500, 1000],
        },
        {
            'penalty': ['l1', 'l2', 'none'],  # l1, l2 and none penalties
            'C': [0.01, 0.1, 1.0, 10.0, 100.0],
            'solver': ['saga'],  # saga supports l1, l2, and none
            'max_iter': [100, 500, 1000],
        }
    ]

    # Create a logistic regression model
    lr = LogisticRegression(random_state=42)

    # Perform grid search over the hyperparameter grids using 5-fold cross-validation
    grid_search = GridSearchCV(estimator=lr,
                               param_grid=param_grids,
                               cv=5,
                               n_jobs=-1)

    # Fit the grid search to the training data
    grid_search.fit(X_train, y_train)

    # Print the best parameters and best score
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best score: {grid_search.best_score_}")
    
    return grid_search.best_params_

def train_best_model(X_train, y_train, params):
    # Create a new logistic regression model with the best hyperparameters
    best_lr = LogisticRegression(**params, random_state=42)

    # Fit the new model to the training data
    best_lr.fit(X_train, y_train)
    
    return best_lr

def evaluate_model(best_lr, X_dev, y_dev):
    # Evaluate the performance of the new model on the test data
    score = best_lr.score(X_dev, y_dev)
    print(f"Test score: {score}")

    # Print the results
    y_pred = best_lr.predict(X_dev)
    print(classification_report(y_dev, y_pred))

    # Create the confusion matrix
    cm = confusion_matrix(y_dev, y_pred)

    # Print the confusion matrix
    print("Confusion Matrix:\n", cm)
    
def best_lr(df, Art):
    # One-hot encoding
    X_train, y_train, X_dev, y_dev, X_test, y_test = one_hot_encoding(df, Art)
    # Best params
    params = lr_best_model(X_train, y_train)
    # Best model
    best_lr = train_best_model(X_train, y_train, params)
    # Evaluate model
    evaluate_model(best_lr, X_dev, y_dev)
    
    return best_lr

# Trains logistic regression based on specific attribute
def categorize_attribute(df, df_attribute, attribute):
    # Join with dummy data
    df_tmp = df.join(df_attribute[attribute])
    # Split data
    X_train, y_train, X_dev, y_dev = train_dev_test_split(df_tmp, attribute)
    # Best params
    params = lr_best_model(X_train, y_train)
    # Best model
    lr = train_best_model(X_train, y_train, params)
    # Evaluate model
    evaluate_model(lr, X_dev, y_dev)
    
    return lr

## 2. Modelling - Logistic Regression
### 2.1 Preparing training and development sets
We split the dataset into training & development and test sets and put the test set aside.

In [4]:
df_train_dev, df_test = train_dev_test_split(df.iloc[:,:-1])

### 2.2 Classification by diet
A second approach is to classify by diet first and then build up subsequent models.

#### 2.2.1 Herbivore vs. carnivore and omnivore

In [5]:
# Add herbivore dummy
df_train_dev['Herbivore'] = (df_train_dev['Diet'] == 'herbivor').astype(int)
# Train and dev data
X_train, y_train, X_dev, y_dev = train_dev_test_split(df_train_dev, 'Herbivore')
# Best params
params = lr_best_model(X_train, y_train)
# Best model
lr_herbivore = train_best_model(X_train, y_train, params)
# Evaluate model
evaluate_model(lr_herbivore, X_dev, y_dev)

Best parameters: {'C': 1.0, 'max_iter': 1000, 'penalty': 'l1', 'solver': 'saga'}
Best score: 0.9746031746031747
Test score: 0.9809523809523809
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        57
           1       0.98      0.98      0.98        48

    accuracy                           0.98       105
   macro avg       0.98      0.98      0.98       105
weighted avg       0.98      0.98      0.98       105

Confusion Matrix:
 [[56  1]
 [ 1 47]]


The results are quite good and promising to move further with this approach.

#### 2.2.2 Carnivore vs. omnivore
Given that we distinguished herbivores from other diets, we now want to differentiate between carnivores and omnivores.

In [6]:
# Filter df for carnivores and omnivors
df_carni_omni = df_train_dev[df_train_dev.Diet != 'herbivor']
df_carni_omni = df_carni_omni.drop('Herbivore', axis=1)
df_carni_omni['Carnivore'] = (df['Diet'] == 'carnivor').astype(int)
# Train and dev data
X_train, y_train, X_dev, y_dev = train_dev_test_split(df_carni_omni, 'Carnivore')
# Best params
params = lr_best_model(X_train, y_train)
# Best model
lr_carnivore = train_best_model(X_train, y_train, params)
# Evaluate model
evaluate_model(lr_carnivore, X_dev, y_dev)

Best parameters: {'C': 0.01, 'max_iter': 100, 'penalty': 'none', 'solver': 'lbfgs'}
Best score: 0.8309243697478992
Test score: 0.8620689655172413
              precision    recall  f1-score   support

           0       0.74      0.95      0.83        21
           1       0.97      0.81      0.88        37

    accuracy                           0.86        58
   macro avg       0.85      0.88      0.86        58
weighted avg       0.89      0.86      0.86        58

Confusion Matrix:
 [[20  1]
 [ 7 30]]


The results are not as good as the herbivore model but still good.

### 2.3 Classification of family

In [7]:
# One-hot encoding of animal family
df_family = pd.get_dummies(df_train_dev.Familie)

In [8]:
for diet in df_train_dev.Diet.unique():
    print(2*'#'+diet+60*'#')
    df_diet = df_train_dev[df_train_dev.Diet == diet]
    df_family = pd.get_dummies(df_diet.Familie)
    for family in df_diet.Familie.unique():
        print(2*'#'+family+60*'#')
        categorize_attribute(df_diet, df_family, family)  

##herbivor############################################################
##Ailuridae############################################################
Best parameters: {'C': 0.01, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Best score: 1.0
Test score: 1.0
              precision    recall  f1-score   support

       False       1.00      1.00      1.00        42
        True       1.00      1.00      1.00         6

    accuracy                           1.00        48
   macro avg       1.00      1.00      1.00        48
weighted avg       1.00      1.00      1.00        48

Confusion Matrix:
 [[42  0]
 [ 0  6]]
##Bovidae############################################################
Best parameters: {'C': 100.0, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
Best score: 0.9362068965517241
Test score: 0.9791666666666666
              precision    recall  f1-score   support

       False       1.00      0.96      0.98        28
        True       0.95      1.00      0.98   

In [9]:
categorize_attribute(df_diet, df_family, family) 

Best parameters: {'C': 0.01, 'max_iter': 100, 'penalty': 'none', 'solver': 'sag'}
Best score: 0.8462450592885375
Test score: 0.8648648648648649
              precision    recall  f1-score   support

       False       0.73      0.80      0.76        10
        True       0.92      0.89      0.91        27

    accuracy                           0.86        37
   macro avg       0.83      0.84      0.83        37
weighted avg       0.87      0.86      0.87        37

Confusion Matrix:
 [[ 8  2]
 [ 3 24]]


LogisticRegression(C=0.01, penalty='none', random_state=42, solver='sag')

### 3.3 Classification by digestion for herbivores

In [10]:
# Filter train dev dataset for herbivores only
df_herbivore = df_train_dev[df_train_dev.Diet == 'herbivor'].drop('Herbivore', axis=1)
# One-hot encoding of digestion
df_digestion = pd.get_dummies(df_herbivore.digestion)

#### 3.3.1 Classification by digestion for herbivores - Foregut

#### 3.3.2 Classification by digestion for herbivores - Foregut ruminant

In [11]:
lr_foregut_r = categorize_attribute(df_herbivore, df_digestion, 'foregut_ruminant')

Best parameters: {'C': 1.0, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
Best score: 0.9928571428571429
Test score: 0.9583333333333334
              precision    recall  f1-score   support

       False       0.92      1.00      0.96        23
        True       1.00      0.92      0.96        25

    accuracy                           0.96        48
   macro avg       0.96      0.96      0.96        48
weighted avg       0.96      0.96      0.96        48

Confusion Matrix:
 [[23  0]
 [ 2 23]]


#### 3.3.3 Classification by digestion for herbivores - Hindgut caecum

#### 3.3.4 Classification by digestion for herbivores - Hindgut colon

In [12]:
lr_hindgut_co = categorize_attribute(df_herbivore, df_digestion, 'hindgut_colon')

Best parameters: {'C': 0.01, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Best score: 0.9859605911330049
Test score: 1.0
              precision    recall  f1-score   support

       False       1.00      1.00      1.00        31
        True       1.00      1.00      1.00        17

    accuracy                           1.00        48
   macro avg       1.00      1.00      1.00        48
weighted avg       1.00      1.00      1.00        48

Confusion Matrix:
 [[31  0]
 [ 0 17]]


#### 3.3.5 Classification by digestion for herbivores -  Simple

In [13]:
lr_simple = categorize_attribute(df_herbivore, df_digestion, 'simple')

Best parameters: {'C': 0.01, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Best score: 1.0
Test score: 0.9791666666666666
              precision    recall  f1-score   support

       False       0.98      1.00      0.99        42
        True       1.00      0.83      0.91         6

    accuracy                           0.98        48
   macro avg       0.99      0.92      0.95        48
weighted avg       0.98      0.98      0.98        48

Confusion Matrix:
 [[42  0]
 [ 1  5]]


#### 3.3.6 Classification by digestion for herbivores -  Ensemble model

In [14]:
def herbivore_digestion_ensemble(x):
    x = x.astype(float)
    res = {'foregut' : lr_foregut.predict_proba([x.values])[0][1],
           'foregut_ruminant' : lr_foregut_r.predict_proba([x.values])[0][1],
           'hindgut_caecum' : lr_hindgut_ca.predict_proba([x.values])[0][1],
           'hindgut_colon' : lr_hindgut_co.predict_proba([x.values])[0][1],
           'simple' : lr_simple.predict_proba([x.values])[0][1]
          }
    
    max_key = max(res, key=lambda k: res[k])
    
    return max_key

#df_herbivore.iloc[:,9:].apply(digestion_prediction, axis=1)


In [15]:
df_herbivore[['digestion','Familie','Diet']].groupby(['digestion','Familie']).count()#.sort_values('Diet', ascending=False)

Diet
digestion        Familie         
foregut_ruminant Bovidae       78
                 Giraffidae    22
hindgut_colon    Equidae       66
simple           Ailuridae     23
                 Canidae        1

### 3.4 Classification of animal family given diet and digestion

#### 3.4.1 Herbivore and foregut

In [16]:
# Familie - Foregut
df_herb_fg = df_train_dev[(df_train_dev.Diet == 'herbivor') & (df_train_dev.digestion == 'foregut')]
# One-hot encoding of digestion
df_herb_fg_familie = pd.get_dummies(df_herb_fg.Familie)

# Funktioniert nicht, weil zu wenig Beobachtungen:
#lr_Hippopotamidae = categorize_attribute(df_herb_fg, df_herb_fg_familie, 'Hippopotamidae')
#lr_Macropodidae = categorize_attribute(df_herb_fg, df_herb_fg_familie, 'Macropodidae')
#lr_Suidae = categorize_attribute(df_herb_fg, df_herb_fg_familie, 'Suidae')
#lr_Tapiridae = categorize_attribute(df_herb_fg, df_herb_fg_familie, 'Tapiridae')

#### 3.4.2 Herbivore and foregut ruminant

In [17]:
# Familie - Foregut ruminant
df_herb_fg_r = df_train_dev[(df_train_dev.Diet == 'herbivor') & (df_train_dev.digestion == 'foregut_ruminant')]
# One-hot encoding of digestion
df_herb_fg_r_familie = pd.get_dummies(df_herb_fg_r.Familie)

# Create model for each family
print('-' * 80 + "\nBovidae\n" + '-' * 80)
lr_Bovidae = categorize_attribute(df_herb_fg_r, df_herb_fg_r_familie, 'Bovidae')
#print('-' * 80 + "\nCamelidae\n" + '-' * 80)
#lr_Camelidae = categorize_attribute(df_herb_fg_r, df_herb_fg_r_familie, 'Camelidae')
#print('-' * 80 + "\nCervidae\n" + '-' * 80)
#lr_Cervidae = categorize_attribute(df_herb_fg_r, df_herb_fg_r_familie, 'Cervidae')
print('-' * 80 + "\nGiraffidae\n" + '-' * 80)
lr_Giraffidae = categorize_attribute(df_herb_fg_r, df_herb_fg_r_familie, 'Giraffidae')

--------------------------------------------------------------------------------
Bovidae
--------------------------------------------------------------------------------
Best parameters: {'C': 1.0, 'max_iter': 500, 'penalty': 'l1', 'solver': 'saga'}
Best score: 0.9333333333333333
Test score: 0.96
              precision    recall  f1-score   support

       False       1.00      0.83      0.91         6
        True       0.95      1.00      0.97        19

    accuracy                           0.96        25
   macro avg       0.97      0.92      0.94        25
weighted avg       0.96      0.96      0.96        25

Confusion Matrix:
 [[ 5  1]
 [ 0 19]]
--------------------------------------------------------------------------------
Giraffidae
--------------------------------------------------------------------------------
Best parameters: {'C': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Best score: 0.8933333333333333
Test score: 1.0
              precision    recal

#### 3.4.3 Herbivore and hindgut colon

In [18]:
# Familie - Hingut colon
df_herb_hg_co = df_train_dev[(df_train_dev.Diet == 'herbivor') & (df_train_dev.digestion == 'hindgut_colon')]
# One-hot encoding of digestion
df_herb_hg_co_familie = pd.get_dummies(df_herb_hg_co.Familie)

# Create model for each family
#print('-' * 80 + "\nElephantidae\n" + '-' * 80)
#lr_Elephantidae = categorize_attribute(df_herb_hg_co, df_herb_hg_co_familie, 'Elephantidae')
print('-' * 80 + "\nEquidae\n" + '-' * 80)
lr_Equidae = categorize_attribute(df_herb_hg_co, df_herb_hg_co_familie, 'Equidae')
#print('-' * 80 + "\nPhascolarctidae\n" + '-' * 80)
#lr_Phascolarctidae = categorize_attribute(df_herb_hg_co, df_herb_hg_co_familie, 'Phascolarctidae')
#print('-' * 80 + "\nRhinocerotidae\n" + '-' * 80)
#lr_Rhinocerotidae = categorize_attribute(df_herb_hg_co, df_herb_hg_co_familie, 'Rhinocerotidae')
#print('-' * 80 + "\nVombatidae\n" + '-' * 80)
#lr_Vombatidae = categorize_attribute(df_herb_hg_co, df_herb_hg_co_familie, 'Vombatidae')

--------------------------------------------------------------------------------
Equidae
--------------------------------------------------------------------------------


ValueError: 
All the 825 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\dmont\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\dmont\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dmont\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1227, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
                                                ^^^^^^^^^^^^^^^
  File "C:\Users\dmont\anaconda3\Lib\site-packages\

#### 3.4.4 Herbivore and simple

In [ ]:
# Familie - Simple
df_herb_simple = df_train_dev[(df_train_dev.Diet == 'herbivor') & (df_train_dev.digestion == 'simple')]
# One-hot encoding of digestion
df_herb_simple_familie = pd.get_dummies(df_herb_simple.Familie)

# Create model for each family
print('-' * 80 + "\nAiluridae\n" + '-' * 80)
lr_Ailuridae = categorize_attribute(df_herb_simple, df_herb_simple_familie, 'Ailuridae')
print('-' * 80 + "\nHomininae\n" + '-' * 80)
lr_Homininae = categorize_attribute(df_herb_simple, df_herb_simple_familie, 'Homininae')
print('-' * 80 + "\nLemuridae\n" + '-' * 80)
lr_Lemuridae = categorize_attribute(df_herb_simple, df_herb_simple_familie, 'Lemuridae')

### 3.5 Ensemble model

In [ ]:
y_dev = df.Art.loc[X_dev.index]

In [ ]:
def foregut_ruminant(microbiome):
    res = {'Bovidae' : lr_Bovidae.predict_proba(microbiome)[0][1],
           'Camelidae' : lr_Camelidae.predict_proba(microbiome)[0][1],
           'Cervidae' : lr_Cervidae.predict_proba(microbiome)[0][1],
           'Giraffidae' : lr_Giraffidae.predict_proba(microbiome)[0][1]}
    
    max_key = max(res, key=lambda k: res[k])
    
    return max_key
    

def categorize_microbiome(microbiome):
    res = []
    # First, predict if herbivore or carni-/omnivore
    diet = lr_herbivore.predict(microbiome)
    # Second, based on diet predict type of digestion
    for i in range(len(diet)):
        if diet[i] == 1:
            digestion = herbivore_digestion_ensemble(microbiome.iloc[i])
            res.append()
        else:
            res.append(metadata_familie[(metadata_familie.Diet != 'herbivor')].Familie.drop_duplicates().to_list())
        
    return res

In [ ]:
x = df.iloc[0,9:-1].astype(float)